## Load Coil20 data
## Extract DSIFT features

In [1]:
import matlab.engine
eng = matlab.engine.start_matlab()
eng.cd("./SSC_ADMM_v1.1")

'C:\\Users\\aleks_000\\Desktop\\Mentorship\\!GitHub'

In [2]:
from load import load_Coil20
from img2matrix import single_img2dsift

# I renamed files 1, 2.. to 01, 02..
# so that they are in order here
images_raw, labels = load_Coil20()
images_dsift = [single_img2dsift(image) for image in images_raw]


Loading Coil20...
----------------
Elapsed: 22.30 sec


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2)

ax[0].imshow(images_raw[0].reshape((-1, 128)));
ax[1].imshow(images_dsift[0].reshape((-1, 128)));

In [ ]:
import numpy as np

fig, ax = plt.subplots(1, 2)

ax[0].imshow(np.mean(images_raw, axis=0).reshape((-1, 128)));
ax[1].imshow(np.mean(images_dsift, axis=0).reshape((-1, 128)));

In [ ]:
fig, ax = plt.subplots(1, 2)
idx = 0

ax[0].imshow(np.mean(images_raw[72*idx:72*(idx+1)], axis=0).reshape((-1, 128)));
ax[1].imshow(np.mean(images_dsift[72*idx:72*(idx+1)], axis=0).reshape((-1, 128)));

In [ ]:
fig, ax = plt.subplots(1, 2)

ax[0].imshow(np.std(images_raw, axis=0).reshape((-1, 128)));
ax[1].imshow(np.std(images_dsift, axis=0).reshape((-1, 128)));

In [7]:
from visualize import tSNE_2D, tSNE_3D

In [8]:
images_raw_flat = images_raw.reshape(images_raw.shape[0], -1)
# tSNE_2D(images_raw_flat, labels)
# tSNE_3D(images_raw_flat, labels)

In [9]:
# tSNE_2D(images_dsift, labels)
# tSNE_3D(images_dsift, labels)

## Perform PCA

In [20]:
from sklearn.decomposition import PCA
import numpy as np

pca = PCA(n_components=1000, whiten=False, svd_solver='arpack', random_state=0)
images_pca = pca.fit_transform(images_raw_flat)

images_pca.shape

(1440, 1000)

In [21]:
data_h = 25
data_w = 40

In [22]:
import matplotlib.pyplot as plt

def display_image(image, height, width):
    print(np.min(image), np.max(image))
    imgplot = plt.imshow(image.reshape((height, width)))

In [ ]:
# these are reduced parameters - not supposed to look like anything
display_image(images_pca[0], data_h, data_w)

In [ ]:
display_image(np.mean(images_pca, axis=0), data_h, data_w)

In [ ]:
display_image(np.std(images_pca, axis=0), data_h, data_w)

In [26]:
# Check reconstruction
pca_reconstr = pca.inverse_transform(images_pca)

In [ ]:
display_image(pca_reconstr[0], 128, 128)

In [28]:
# tSNE_2D(images_pca, labels)
# tSNE_3D(images_pca, labels)

## Normalize PCA output

In [29]:
# Types:
# feature-wise - normalization occurs along one pixel of all images
# image-wise - normalization occurs along all pixels of one image
# all - normalization occurs along all pixels of all images

# Methods:
# standard - mean is set to 0, std is set to 1
# [-1, 1] - min and max are used to linearly change the data range to [-1, 1]
# unit-vector - each bin is divided by its euclidean distance

#[BAD] feature, standard
#images_norm = (images_pca - np.mean(images_pca, axis=0)) / np.std(images_pca, axis=0)
# image, standard
#images_norm = (images_pca - np.mean(images_pca, axis=1)[:, np.newaxis]) / np.std(images_pca, axis=1)[:, np.newaxis]
# all, standard
#images_norm = (images_pca - np.mean(images_pca)) / np.std(images_pca)


#[BAD] feature, [-1, 1]
#mmin = np.min(images_pca, axis=0)
#mmax = np.max(images_pca, axis=0)
#[BAD] image, [-1, 1]
#mmin = np.min(images_pca, axis=1)[:, np.newaxis]
#mmax = np.max(images_pca, axis=1)[:, np.newaxis]
# all, [-1, 1]
mmin = np.min(images_pca)
mmax = np.max(images_pca)
# FOR ALL:
images_norm = (np.multiply(images_pca, 2, dtype='float64') - mmax - mmin) / (mmax - mmin)

#[BAD] feature, unit
#images_norm = images_pca / np.sqrt(np.sum(images_pca*images_pca, axis=0))
# image, unit
#images_norm = images_pca / np.sqrt(np.sum(images_pca*images_pca, axis=1))[:, np.newaxis]
# all, unit
#images_norm = images_pca / np.sqrt(np.sum(images_pca*images_pca))

In [30]:
# Only run when using something other than images_pca for images_norm
#data_h = 128
#data_w = 128

In [ ]:
import matplotlib.pyplot as plt

# these are reduced parameters - not supposed to look like anything
display_image(images_norm[0], data_h, data_w)

In [ ]:
display_image(np.mean(images_norm, axis=0), data_h, data_w)

In [ ]:
stds = np.std(images_norm, axis=0)
display_image(stds, data_h, data_w)

In [51]:
print("Average std:", np.mean(stds))
print("By rows:", np.mean(stds.reshape((10, -1)), axis=1))

Average std: 0.014709905614717902
By rows: [0.06997777 0.0198563  0.0133854  0.01029025 0.00828885 0.00683101
 0.00574123 0.00489309 0.00420903 0.00362612]


In [34]:
# tSNE_2D(images_norm, labels)
# tSNE_3D(images_norm, labels)

In [35]:
# from scipy.io import savemat

# savemat('./saved/raw/Coil20.mat', mdict={'X':images_raw, 'Y':labels})
# savemat('./saved/processed/Coil20.mat', mdict={'X':images_norm, 'Y':labels})

## Hyperparameters

In [36]:
lamb1 = 0.0
lamb2 = 0.00508823385491712
lamb3 = 0.0

## Calculate C matrix

In [37]:
import numpy as np
import supporting_files.sda as sda

from supporting_files.helpers import optimize
from scipy.io import savemat, loadmat

In [38]:
# Matlab SSC #1
savemat('./temp.mat', mdict={'X': images_norm})
k = len(np.unique(labels))
alpha = 10.0
maxIter = 63
eng.SSC_modified(k, 0, False, alpha, False, 1, 1e-20, maxIter, False, 0)
C = loadmat("./temp.mat")['C']

In [ ]:
display_image(C[:110, :110], 110, 110)
print(np.mean(np.square(C)))

In [ ]:
fig, ax = plt.subplots(2)

index = 0;
ax[0].imshow(images_norm[index].reshape((data_h, data_w)));
ax[1].imshow(np.matmul(C, images_norm)[index].reshape((data_h, data_w)));
print(np.mean(np.square(images_norm - np.matmul(C, images_norm))))

## Train Autoencoder

In [41]:
import dsc
import importlib
importlib.reload(dsc)

<module 'dsc' from 'C:\\Users\\aleks_000\\Desktop\\Mentorship\\!GitHub\\dsc.py'>

In [103]:
%%time
trainC = False
C = None
d = dsc.DeepSubspaceClustering(images_norm, C=C, trainC=trainC, hidden_dims=[300, 150, 300],
                               lambda1=lamb1, lambda2=lamb2, lambda3=lamb3, weight_init='sda-normal',
                               weight_init_params=[1000, 0.0005, images_norm.shape[0], 500],
                               optimizer='Adam', decay='sqrt', sda_optimizer='Adam', sda_decay='sqrt')

Layer 1
1000 -> 300
epoch 0: global loss = 0.12220864742994308
epoch 500: global loss = 0.02138262242078781
Layer 2
300 -> 150
epoch 0: global loss = 0.1676410734653473
epoch 500: global loss = 0.024683743715286255
Wall time: 1min 27s


In [104]:
encoded_h = 10
encoded_w = 15

In [105]:
%%time
d.optimizer = optimize(d.cost, 0.0001, 'Adam', 'sqrt', d.global_step)
d.train(batch_size=images_norm.shape[0], epochs=500, print_step=100)
images_HM2 = d.result
images_HM = d.reconstr
if(trainC):
    trained_C = np.float64(d.outC)
else:
    trained_C = C


epoch 0: global loss = 0.0008810296421870589
epoch 100: global loss = 0.000527861644513905
epoch 200: global loss = 0.0005129497731104493
epoch 300: global loss = 0.0005015017231926322
epoch 400: global loss = 0.0004919956554658711
Wall time: 1min 2s


In [106]:
# lambda3 - regularization on trained_C
if(C is not None):
    display_image(trained_C[:110, :110], 110, 110)

In [ ]:
display_image(images_HM2[0], encoded_h, encoded_w)

In [108]:
# lambda1 - self-expressiveness
if(C is not None):
    display_image(np.matmul(trained_C, images_HM2)[0], encoded_h, encoded_w)
    print(np.mean(np.square(images_HM2 - np.matmul(trained_C, images_HM2))))

In [ ]:
display_image(np.mean(images_HM2, axis=0), encoded_h, encoded_w)

In [ ]:
display_image(np.std(images_HM2, axis=0), encoded_h, encoded_w)

In [111]:
# tSNE_2D(images_HM2, labels)
# tSNE_3D(images_HM2, labels)

In [ ]:
# AE Reconstruction
fig, ax = plt.subplots(1, 2)

index = 0;
ax[0].imshow(images_norm[index].reshape((data_h, data_w)));
ax[1].imshow(images_HM[index].reshape((data_h, data_w)));

In [113]:
reconstr = pca.inverse_transform(((images_HM * (mmax - mmin)) + mmax + mmin) / 2)
pca_reconstr2 = pca.inverse_transform(((images_norm * (mmax - mmin)) + mmax + mmin) / 2)

In [ ]:
# AE + invPCA Reconstruction
fig, ax = plt.subplots(1, 3)

index = 0;
#ax[0].imshow(images_dsift[index].reshape((-1, 128)));
ax[0].imshow(images_raw_flat[index].reshape((-1, 128)));
ax[1].imshow(pca_reconstr2[index].reshape((-1, 128)));
ax[2].imshow(reconstr[index].reshape((-1, 128)));

In [ ]:
from ipywidgets import BoundedIntText, FloatSlider, Output, VBox
idx = BoundedIntText(description="Index:", max=images_HM.shape[1]-1)
val = FloatSlider(description="Value:", continuous_update=False)
output = Output()

vector = images_HM[0].copy()
val_mins = (np.mean(images_HM, axis=0) - 2 * np.std(images_HM, axis=0)).flatten()
val_maxs = (np.mean(images_HM, axis=0) + 2 * np.std(images_HM, axis=0)).flatten()

def update_channels(change):
    val.min = val_mins[change.new]
    val.max = val_maxs[change.new]
    val.step = (val_maxs[change.new] - val_mins[change.new]) / 100
    val.value = vector[change.new]
idx.observe(update_channels, 'value')

def update_plot(change):
    if(change is not None):
        vector[idx.value] = change.new
    output.clear_output(wait=True)
    with output:
        plt.imshow(vector.reshape((data_h, data_w)))
        plt.show()
        #plt.imshow((pca.inverse_transform((vector * (mmax - mmin)) + mmax + mmin) / 2).reshape((-1, 128)))
        #plt.show()
val.observe(update_plot, 'value')

update_channels(type('obj', (object,), {'new': 0}))
VBox([idx, val, output])

In [116]:
# Matlab SSC #2
k = len(np.unique(labels))
alpha = 20.0
maxIter = 63
if(not trainC):
    savemat('./temp.mat', mdict={'X': images_HM2})
else:
    savemat('./temp.mat', mdict={'C': C})
grps = eng.SSC_modified(k, 0, False, alpha, False, 1, 1e-20, maxIter, True, 0, trainC)
C_after = loadmat("./temp.mat")['C']
labels_pred = np.asarray(grps, dtype=np.int32).flatten()

In [ ]:
display_image(C_after[:110, :110], 110, 110)

In [101]:
# tSNE_2D(images_HM2, labels_pred)
# tSNE_3D(images_HM2, labels_pred)

## Perform clustering with SSC

In [118]:
from supporting_files.ji_zhang import err_rate
from sklearn.metrics import normalized_mutual_info_score as nmi
from sklearn.metrics import adjusted_rand_score as ari

print("Accuracy: ", str(1-err_rate(labels, labels_pred)))
print("NMI: ", str(nmi(labels, labels_pred, average_method="geometric")))
print("ARI: ", str(ari(labels, labels_pred)))

Accuracy:  0.5055555555555555
NMI:  0.6839713390951989
ARI:  0.42915980360113415
